# Deep Learning with keras

In this workshop, we will try to build some feedforward models to do sentiment analysis, using keras, a deep learning library: https://keras.io/

You will need pandas, keras (2.3.1) and tensorflow (2.1.0; and their dependencies) to run this code (pip install pandas keras==2.3.1 tensorflow-cpu==2.1.0).

First let's prepare the data. We are using 1000 yelp reviews, nnotated with either positive or negative sentiments.

In [1]:
import pandas as pd

corpus = "07-yelp-dataset.txt"
df = pd.read_csv(corpus, names=['sentence', 'label'], sep='\t')
print("Number of sentences =", len(df))
print("\nData:")
print(df.iloc[:5])

Number of sentences = 1000

Data:
                                            sentence  label
0                           Wow... Loved this place.      1
1                                 Crust is not good.      0
2          Not tasty and the texture was just nasty.      0
3  Stopped by during the late May bank holiday of...      1
4  The selection on the menu was great and so wer...      1


Next, let's create the train/dev/test partitions

In [2]:
import random
import numpy as np

sentences = df['sentence'].values
labels = df['label'].values

#partition data into 80/10/10 for train/dev/test
sentences_train, y_train = sentences[:800], labels[:800]
sentences_dev, y_dev = sentences[800:900], labels[800:900]
sentences_test, y_test = sentences[900:1000], labels[900:1000]

#convert label list into arrays
y_train = np.array(y_train)
y_dev = np.array(y_dev)
y_test = np.array(y_test)

print(y_train[0], sentences_train[0])
print(y_dev[0], sentences_dev[0])
print(y_test[0], sentences_test[0])

1 Wow... Loved this place.
0 I'm super pissd.
0 Spend your money elsewhere.


Let's tokenize the text. In this workshop, we'll use the ``tokenizer`` function provided by keras. Once the data is tokenized, we can then use ``texts_to_matrix`` to get the bag-of-words representation for each document.

In [3]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token="<UNK>")
tokenizer.fit_on_texts(sentences_train)

x_train = tokenizer.texts_to_matrix(sentences_train, mode="count") #BOW representation
x_dev = tokenizer.texts_to_matrix(sentences_dev, mode="count") #BOW representation
x_test = tokenizer.texts_to_matrix(sentences_test, mode="count") #BOW representation

vocab_size = x_train.shape[1]
print(x_train.shape)
print("Vocab size =", vocab_size)
print(x_train[0])

Using TensorFlow backend.


(800, 1811)
Vocab size = 1811
[0. 0. 0. ... 0. 0. 0.]


Before we build a neural network model, let's see how well logistic regression do with this dataset.

In [4]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
score = classifier.score(x_test, y_test)

print("Accuracy:", score)

Accuracy: 0.69


The logistic regression result is not too bad, and it will serve as a baseline for the deep learning models.

Now let's build a very simple feedforward network. Here the input layer is the BOW features, and we have one hidden layer (dimension = 10) and an output layer in the model.

In [5]:
from keras.models import Sequential
from keras import layers

#model definition
model = Sequential(name="feedforward-bow-input")
model.add(layers.Dense(10, input_dim=vocab_size, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

#since it's a binary classification problem, we use a binary cross entropy loss here
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "feedforward-bow-input"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                18120     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 18,131
Trainable params: 18,131
Non-trainable params: 0
_________________________________________________________________


Let's train the model. Notice that there are a few hyper-parameters here, e.g. hidden layer size, number of epochs and batch_size, and in practice these hyper-parameters should be tuned according to the development data to get an optimal model. In this workshop we'll use 20 epochs and a batch size of 10 (no further tuning). Once the model is trained, we'll compute the test accuracy performance.

In [6]:
#training
model.fit(x_train, y_train, epochs=20, verbose=True, validation_data=(x_dev, y_dev), batch_size=10)

loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
print("\nTesting Accuracy:  {:.4f}".format(accuracy))

Train on 800 samples, validate on 100 samples
Epoch 1/20
800/800 [==============================] - 0s 432us/step - loss: 0.6835 - accuracy: 0.5738 - val_loss: 0.6771 - val_accuracy: 0.5800
Epoch 2/20
800/800 [==============================] - 0s 99us/step - loss: 0.6279 - accuracy: 0.7775 - val_loss: 0.6404 - val_accuracy: 0.6100
Epoch 3/20
800/800 [==============================] - 0s 92us/step - loss: 0.5372 - accuracy: 0.8512 - val_loss: 0.5832 - val_accuracy: 0.6900
Epoch 4/20
800/800 [==============================] - 0s 95us/step - loss: 0.4307 - accuracy: 0.9100 - val_loss: 0.5231 - val_accuracy: 0.7500
Epoch 5/20
800/800 [==============================] - 0s 91us/step - loss: 0.3380 - accuracy: 0.9463 - val_loss: 0.4890 - val_accuracy: 0.7400
Epoch 6/20
800/800 [==============================] - 0s 89us/step - loss: 0.2669 - accuracy: 0.9650 - val_loss: 0.4549 - val_accuracy: 0.7800
Epoch 7/20
800/800 [==============================] - 0s 84us/step - loss: 0.2124 - accuracy: 0

How does the performance compare to logistic regression? If you run it a few times you may find that it gives slightly different numbers, and that is due to random initialisation of the model parameters.

Even though we did not explicitly define any word embeddings in the model architecture, they are in our model: in the weights between the input and the hidden layer. The hidden layer can therefore be interpreted as a sum of word embeddings for each input document.

Let's fetch the word embeddings of some words, and look at their cosine similarity, and see if they make any sense.

In [7]:
from numpy import dot
from numpy.linalg import norm

embeddings = model.get_layer(index=0).get_weights()[0] #word embeddings layer
print(len(embeddings[0]))

emb_love = embeddings[tokenizer.word_index["love"]] #embeddings for 'love'
emb_like = embeddings[tokenizer.word_index["like"]]
emb_lukewarm = embeddings[tokenizer.word_index["lukewarm"]]
emb_bad = embeddings[tokenizer.word_index["bad"]]

print(emb_love)

def cos_sim(a, b):
    return dot(a, b)/(norm(a)*norm(b))

print("love vs. like =", cos_sim(emb_love, emb_like))
print("love vs. lukewarm =", cos_sim(emb_love, emb_lukewarm))
print("love vs. bad =", cos_sim(emb_love, emb_bad))
print("lukewarm vs. bad =", cos_sim(emb_lukewarm, emb_bad))

10
[ 0.23626566  0.35327494  0.2273566   0.2975782  -0.23939022 -0.2554149
 -0.2522671  -0.17758775 -0.2757556  -0.18357718]
love vs. like = 0.7028187
love vs. lukewarm = -0.97351277
love vs. bad = -0.96983844
lukewarm vs. bad = 0.956362


Not bad. You should find that for *love* and *like*, which are both positive sentiment words, produce high cosine similarity. Similar observations for *lukewarm* and *bad*. But when we compare opposite polarity words like *love* and *bad*, we get negative cosine similarity values.

Next, we are going to build another feed-forward model, but this time, instead of using BOW features as input, we want to use the word sequence as input (so order of words is preserved). It is usually not straightforward to do this for classical machine learning models, but with neural networks and embeddings, it's pretty straightforward.

Let's first tokenise the input documents into word sequences.

In [8]:
#tokenise the input into word sequences

xseq_train = tokenizer.texts_to_sequences(sentences_train)
xseq_dev = tokenizer.texts_to_sequences(sentences_dev)
xseq_test = tokenizer.texts_to_sequences(sentences_test)

print(sentences_train[:5])
print(xseq_train[:5])

['Wow... Loved this place.' 'Crust is not good.'
 'Not tasty and the texture was just nasty.'
 'Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.'
 'The selection on the menu was great and so were the prices.']
[[354, 138, 9, 17], [475, 8, 13, 16], [13, 123, 3, 2, 476, 4, 48, 477], [478, 68, 299, 2, 479, 355, 762, 763, 185, 764, 765, 356, 3, 138, 10], [2, 139, 32, 2, 108, 4, 21, 3, 23, 27, 2, 140]]


Because documents have variable lengths, we need to first 'pad' them to make all documents have the same length. keras uses word index 0 to represent 'pad symbols'.

In [9]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 30
xseq_train = pad_sequences(xseq_train, padding='post', maxlen=maxlen)
xseq_dev = pad_sequences(xseq_dev, padding='post', maxlen=maxlen)
xseq_test = pad_sequences(xseq_test, padding='post', maxlen=maxlen)
print(xseq_train[0])
print(xseq_train.shape)

[354 138   9  17   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
(800, 30)


Now let's build our second model. This model first embeds each word in the input sequence into embeddings, and then concatenate the word embeddings together to represent input sequence. The ``Flatten`` function you see after the embedding layer is essentially doing the concatenation, by 'chaining' the list of word embeddings into a very long vector.

If our word embeddings has a dimension 10, and our documents always have 30 words (padded), then here the concatenated word embeddings have a dimension of 10 x 30 = 300. 

The concatenated word embeddings undergo a linear transformation with non-linear activations (``layers.Dense(10, activation='relu')``), producing a hidden representation with a dimension of 10. It is then passed to the output layer.

In [10]:
embedding_dim = 10

#word order preserved with this architecture
model2 = Sequential(name="feedforward-sequence-input")
model2.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model2.add(layers.Flatten())
model2.add(layers.Dense(10, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model2.summary()

Model: "feedforward-sequence-input"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 10)            18110     
_________________________________________________________________
flatten_1 (Flatten)          (None, 300)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                3010      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 21,131
Trainable params: 21,131
Non-trainable params: 0
_________________________________________________________________


In [17]:
xseq_train.shape

(800, 30)

Now let's train the model and compute the test accuracy.

In [12]:
model2.fit(xseq_train, y_train, epochs=20, verbose=True, validation_data=(xseq_dev, y_dev), batch_size=10)

loss, accuracy = model2.evaluate(xseq_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 800 samples, validate on 100 samples
Epoch 1/20
800/800 [==============================] - 0s 297us/step - loss: 0.6881 - accuracy: 0.5663 - val_loss: 0.7021 - val_accuracy: 0.4400
Epoch 2/20
800/800 [==============================] - 0s 91us/step - loss: 0.6708 - accuracy: 0.6175 - val_loss: 0.7099 - val_accuracy: 0.4800
Epoch 3/20
800/800 [==============================] - 0s 85us/step - loss: 0.6197 - accuracy: 0.7275 - val_loss: 0.6781 - val_accuracy: 0.5700
Epoch 4/20
800/800 [==============================] - 0s 86us/step - loss: 0.4929 - accuracy: 0.8737 - val_loss: 0.6926 - val_accuracy: 0.5600
Epoch 5/20
800/800 [==============================] - 0s 92us/step - loss: 0.3194 - accuracy: 0.9400 - val_loss: 0.6139 - val_accuracy: 0.6700
Epoch 6/20
800/800 [==============================] - 0s 88us/step - loss: 0.1896 - accuracy: 0.9675 - val_loss: 0.6002 - val_accuracy: 0.6400
Epoch 7/20
800/800 [==============================] - 0s 86us/step - loss: 0.1224 - accuracy: 0

You may find that the performance isn't as good as the BOW model. In general, concatenating word embeddings isn't a good way to represent word sequence.

A better way is to build a recurrent model. But first, let's extract the word embeddings for the 4 words as before and look at their similarity.

In [13]:
embeddings = model2.get_layer(index=0).get_weights()[0] #word embeddings

emb_love = embeddings[tokenizer.word_index["love"]]
emb_like = embeddings[tokenizer.word_index["like"]]
emb_lukewarm = embeddings[tokenizer.word_index["lukewarm"]]
emb_bad = embeddings[tokenizer.word_index["bad"]]

print("love vs. like =", cos_sim(emb_love, emb_like))
print("love vs. lukewarm =", cos_sim(emb_love, emb_lukewarm))
print("love vs. bad =", cos_sim(emb_love, emb_bad))
print("lukewarm vs. bad =", cos_sim(emb_lukewarm, emb_bad))

love vs. like = 0.45001945
love vs. lukewarm = 0.2256921
love vs. bad = -0.3167041
lukewarm vs. bad = 0.14528398


In [19]:
embeddings.shape

(1811, 10)

Now, let's try to build an LSTM model. After the embeddings layer, the LSTM layer will process the words one at a time, and compute the next state (dimension for the hidden state = 10 in this case). The output of the LSTM layer is the final state, produced after processing the last word, and that will be fed to the output layer.

In [14]:
from keras.layers import LSTM

#word order preserved with this architecture
model3 = Sequential(name="lstm")
model3.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model3.add(LSTM(10))
model3.add(layers.Dense(1, activation='sigmoid'))
model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model3.summary()

Model: "lstm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 10)            18110     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 18,961
Trainable params: 18,961
Non-trainable params: 0
_________________________________________________________________


Let's train the LSTM model and see the test performance.

In [15]:
model3.fit(xseq_train, y_train, epochs=20, verbose=True, validation_data=(xseq_dev, y_dev), batch_size=10)

loss, accuracy = model3.evaluate(xseq_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 800 samples, validate on 100 samples
Epoch 1/20
800/800 [==============================] - 1s 2ms/step - loss: 0.6869 - accuracy: 0.5650 - val_loss: 0.7164 - val_accuracy: 0.4400
Epoch 2/20
800/800 [==============================] - 1s 963us/step - loss: 0.6851 - accuracy: 0.5650 - val_loss: 0.7236 - val_accuracy: 0.4400
Epoch 3/20
800/800 [==============================] - 1s 981us/step - loss: 0.6697 - accuracy: 0.5713 - val_loss: 0.6789 - val_accuracy: 0.4700
Epoch 4/20
800/800 [==============================] - 1s 926us/step - loss: 0.4216 - accuracy: 0.8512 - val_loss: 0.5176 - val_accuracy: 0.7900
Epoch 5/20
800/800 [==============================] - 1s 955us/step - loss: 0.2429 - accuracy: 0.9287 - val_loss: 0.6174 - val_accuracy: 0.7800
Epoch 6/20
800/800 [==============================] - 1s 913us/step - loss: 0.1598 - accuracy: 0.9575 - val_loss: 0.5024 - val_accuracy: 0.8100
Epoch 7/20
800/800 [==============================] - 1s 975us/step - loss: 0.1067 - accurac

You should notice that the training is quite a bit slower, and that's because now the model has to process the sequence one word at a time. But the results should be better!

And lastly, let's extract the embeddings and look at the their similarity.

In [16]:
embeddings = model3.get_layer(index=0).get_weights()[0] #word embeddings

emb_love = embeddings[tokenizer.word_index["love"]]
emb_like = embeddings[tokenizer.word_index["like"]]
emb_lukewarm = embeddings[tokenizer.word_index["lukewarm"]]
emb_bad = embeddings[tokenizer.word_index["bad"]]

print("love vs. like =", cos_sim(emb_love, emb_like))
print("love vs. lukewarm =", cos_sim(emb_love, emb_lukewarm))
print("love vs. bad =", cos_sim(emb_love, emb_bad))
print("lukewarm vs. bad =", cos_sim(emb_lukewarm, emb_bad))

love vs. like = 0.59158075
love vs. lukewarm = -0.83444756
love vs. bad = -0.9356726
lukewarm vs. bad = 0.8281923
